In [1]:
## IMPORT DAS BIBLIOTECAS
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
load_dotenv('OPEN_API_KEY.env')

True

In [2]:
## CHAMANDO O CHAT DO GPT
chat = ChatOpenAI(
    model = 'gpt-3.5-turbo',
)

In [3]:
## CRIANDO UM MODELO BÁSICO DE RESPOSTA
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage,
)

messages = [
    SystemMessage(content='Você é um assitente biblico'),
    HumanMessage(content='Olá Bot, como você está hoje?'),
    AIMessage(content='Estou bem, obirgado! Como posso te ajudar?'),
]

In [4]:
## CRIANDO UM MODELO BÁSICO DE RESPOSTA
prompt = HumanMessage(
    content='Quantos filhos Ramás teve?'
)
## Trazer as informações de acordo com o histórico!
messages.append(prompt)

In [5]:
## RETORNANDO A INFORMAÇÃO
res = chat.invoke(messages)
print(res.content)

Segundo 1 Crônicas 2:10-11, Ramá teve dois filhos chamados Arão e Samuel.


In [6]:
## COMEÇANDO A ENTENDER O RAG
INFORMACOES_UTEIS = ['O Yago comeu ontem arroz, batata e feijão ele gostou bastante da comida']
source_knowledge = "\n".join(INFORMACOES_UTEIS)
query = "Você pode me falar o que o Yago comeu ontem?"
augmented_prompt = f"""Use o contexto abaixo para responder.

Contexto:
{source_knowledge}

Pergunta:
{query}
"""


In [7]:
## COMEÇANDO A ENTENDER O RAG
prompt = HumanMessage(
    content=augmented_prompt
)

messages.append(prompt)

res= chat.invoke(messages)
print(res.content)

Sim, o Yago comeu arroz, batata e feijão e gostou bastante da comida.


RAG

In [8]:
## IMPORTANDO A BASE DE DADOS
import pandas as pd
df = pd.read_excel('clubes_campeoes_brasileirao.xlsx')
df

,Clube Campeão,Número de Títulos,História
0,Palmeiras,11,"Fundado em 1914 como Palestra Italia, em São P..."
1,Santos,8,"Fundado em 1912, em Santos, destacou-se na era..."
2,Corinthians,7,"Criado em 1910, em São Paulo, é um dos clubes ..."
3,Flamengo,8,"Fundado em 1895, no Rio de Janeiro, é um dos m..."
4,São Paulo,6,"Fundado em 1930, em São Paulo, vencedor de 6 e..."
5,Cruzeiro,4,"Criado em 1921, em Belo Horizonte, conquistou ..."
6,Fluminense,4,"Fundado em 1902, no Rio de Janeiro, conquistou..."
7,Vasco da Gama,4,"Criado em 1898, no Rio de Janeiro, tem 4 campe..."
8,Atlético Mineiro,2,"Fundado em 1908, em Minas Gerais, faturou 2 tí..."
9,Internacional,3,"Criado em 1909, no Rio Grande do Sul, venceu 3..."


In [9]:
##Transformo o df para o estilo de documento do Langchain
from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(df, page_content_column='História')
documents = loader.load()

In [10]:
##Personalizando o texto em um padrão
from langchain.schema import Document
documents = []

for _, row in df.iterrows():
    content = f"""Clube: {row['Clube Campeão']}
História: {row['História']}"""
    
    documents.append(Document(page_content=content, metadata=row.to_dict()))

In [11]:
## Trazendo as variáveis do Qdrant
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

qdrant = Qdrant.from_documents(
documents=documents,
embedding=embeddings,
location=":memory:",
collection_name = "chatbot"
)

In [12]:
## Pergunta ao modelo
query = "Quais os 3 clubes com mais títulos?"
qdrant.similarity_search(query, k=5)

[Document(metadata={'Clube Campeão': 'São Paulo', 'Número de Títulos': 6, 'História': 'Fundado em 1930, em São Paulo, vencedor de 6 edições e de 3 Mundiais de Clubes.', '_id': 'b89ebdf68a4f45ba83cf51ca606d1f10', '_collection_name': 'chatbot'}, page_content='Clube: São Paulo\nHistória: Fundado em 1930, em São Paulo, vencedor de 6 edições e de 3 Mundiais de Clubes.'),
 Document(metadata={'Clube Campeão': 'Corinthians', 'Número de Títulos': 7, 'História': 'Criado em 1910, em São Paulo, é um dos clubes mais populares do Brasil, com 7 taças do Brasileirão.', '_id': '4e97bc8b481245dfb0554f3a6caaa330', '_collection_name': 'chatbot'}, page_content='Clube: Corinthians\nHistória: Criado em 1910, em São Paulo, é um dos clubes mais populares do Brasil, com 7 taças do Brasileirão.'),
 Document(metadata={'Clube Campeão': 'Internacional', 'Número de Títulos': 3, 'História': 'Criado em 1909, no Rio Grande do Sul, venceu 3 edições e tem forte tradição no futebol sul-americano.', '_id': '8a778466b35f473

In [13]:
## Propriedades básicas de consulta ao modelo
def custom_prompt(query: str):
    results = qdrant.similarity_search(query, k=17)
    source_knowledge = "\n".join([x.page_content for x in results])
    augmented_prompt= f""" Use o contexto abixo para responder à pergunta.

    Contexto:
    {source_knowledge}

    Pergunta:
    {query}"""
    return augmented_prompt

In [ ]:
## Usando o modelo do GROQ
from langchain_groq import ChatGroq
load_dotenv("GROQ_API_KEY.env")
chat = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [ ]:
## Invocando o modelo
prompt = HumanMessage(
    content=custom_prompt(query)
)

messages.append(prompt)
res = chat.invoke(messages)
print(res.content)

De acordo com o contexto, os 3 clubes com mais títulos são:

1. Palmeiras (11 títulos)
2. Flamengo (8 títulos)
3. Santos (8 títulos)
